In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict
from tqdm.auto import tqdm

import plotly.express as px

# import amp_pd_peptide

from scipy.optimize import minimize

In [ ]:
train_clinical_all = pd.read_csv('./data/train_clinical_data.csv')
proteins = pd.read_csv('./data/train_proteins.csv')

proteins_features = pd.pivot_table(proteins, values='NPX', index='visit_id', columns='UniProt', aggfunc='sum')

train_clinical_all = train_clinical_all.merge(
    proteins_features,
    left_on='visit_id',
    right_index=True,
    how='left'
)

train_clinical_all[proteins_features.columns] = train_clinical_all.groupby('patient_id')[proteins_features.columns].fillna(method='ffill')
display(train_clinical_all)

In [ ]:
train_clinical_all['pred_month'] = train_clinical_all['visit_month']

for plus_month in [6, 12, 24]:
    train_shift = train_clinical_all[['patient_id', 'visit_month', 'pred_month', 'updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']].copy()
    train_shift['visit_month'] -= plus_month
    train_shift.rename(columns={f'updrs_{i}': f'updrs_{i}_plus_{plus_month}' for i in range(1, 5)}, inplace=True)
    train_shift.rename(columns={'pred_month': f'pred_month_plus_{plus_month}'}, inplace=True)
    train_clinical_all = train_clinical_all.merge(train_shift, how='left', on=['patient_id', 'visit_month'])

train_clinical_all.rename(columns={f'updrs_{i}': f'updrs_{i}_plus_0' for i in range(1, 5)}, inplace=True)
train_clinical_all.rename(columns={'pred_month': f'pred_month_plus_0'}, inplace=True)
train_clinical_all

In [ ]:
def smape_plus_1(y_true, y_pred):
    y_true_plus_1 = y_true + 1
    y_pred_plus_1 = y_pred + 1
    metric = np.zeros(len(y_true_plus_1))
    
    numerator = np.abs(y_true_plus_1 - y_pred_plus_1)
    denominator = ((np.abs(y_true_plus_1) + np.abs(y_pred_plus_1)) / 2)
    
    mask_not_zeros = (y_true_plus_1 != 0) | (y_pred_plus_1 != 0)
    metric[mask_not_zeros] = numerator[mask_not_zeros] / denominator[mask_not_zeros]
    
    return 100 * np.nanmean(metric)

def calculate_predictions(pred_month, trend, target):
    if target == 'updrs_4':
        pred_month = pred_month.clip(54, None)
    if len(trend) == 2:
        return np.round(trend[0] + pred_month * trend[1]) # linear prediction
    return np.round(trend[0] + pred_month * trend[1] + np.square(pred_month) * trend[2]) # quadratic prediction

def calculate_predicitons_protein(protein, pred_month, protein_shift):
    trend_pred_month = target_to_trend[target]
    pred_month_trend = calculate_month_trend_predicitons(pred_month=pred_month, trend=trend_pred_month)
    return np.round(pred_month_trend + protein_shift)

def function_to_minimize_trend(x, y_true_array_tr, pred_month_array_tr, target):
    metric = smape_plus_1(
        y_true=y_true_array_tr,
        y_pred=calculate_predictions(
            pred_month=pred_month_array_tr,
            trend=x,
            target=target
        )
    )
    return metric

def function_to_minimize_shift(x, y_true_array_tr, pred_month_array_tr, target):
    metric = smape_plus_1(
        y_true=y_true_array,
        y_pred=calculate_predicitons_protein(
            protein=protein_array,
            pred_month=pred_month_array,
            protein_shift=x[0]
        )
    )
    return metric

def model(y_true_array_tr, pred_month_array_tr, target):
    """Fits a linear or quadratic model to the given data"""
    return list(minimize(
        fun=partial(function_to_minimize_trend,
                    y_true_array_tr=y_true_array_tr,
                    pred_month_array_tr=pred_month_array_tr,
                    target=target
                   ),
        # if x0 has two elements, the predictions will be linear
        # if x0 has three elements, the predictions will be quadratic
        x0=[0, 0.0048] if target != 'updrs_3' else [0, 0, 0],
        method='Powell'
    ).x)

In [ ]:
def find_best_const(train_clinical_all_filtered, target):
    columns_with_target = [f'{target}_plus_{plus_month}' for plus_month in [0, 6, 12, 24]]
    columns_with_pred_month = [f'pred_month_plus_{plus_month}' for plus_month in [0, 6, 12, 24]]
    global y_true_array
    global pred_month_array
    global protein_array
    y_true_array = train_clinical_all_filtered[columns_with_target].values.ravel()
    pred_month_array = train_clinical_all_filtered[columns_with_pred_month].values.ravel()
    protein_array = np.concatenate([train_clinical_all_filtered[feature].values] * 4)
    result = minimize(
        fun=function_to_minimize_shift,
        x0=[0.0],
        method='Powell'
    ).x[0]
    return result

In [ ]:
target_to_trend = {
    'updrs_1': [5.394793062665313, 0.027091086167821344],
    'updrs_2': [5.469498130092747, 0.02824188329658148],
    'updrs_3': [21.182145576879183, 0.08897763331790556],
    'updrs_4': [-4.434453480103724, 0.07531448585334258]
}

In [ ]:
all_score_list = []
for i in range(1, 5):
    target = f'updrs_{i}'
    columns_with_target = [f'{target}_plus_{plus_month}' for plus_month in [0, 6, 12, 24]]
    columns_with_pred_month = [f'pred_month_plus_{plus_month}' for plus_month in [0, 6, 12, 24]]
    kf = GroupKFold(n_splits=5)
    score_list = []
    for f, (idx_tr, idx_va) in enumerate(kf.split(train_clinical_all, groups=train_clinical_all.patient_id)):
        y_true_array_tr = train_clinical_all.iloc[idx_tr][columns_with_target].values.ravel()
        pred_month_array_tr = train_clinical_all.iloc[idx_tr][columns_with_pred_month].values.ravel()
        trend = model(y_true_array_tr, pred_month_array_tr, target)
        
        y_true_array_va = train_clinical_all.iloc[idx_va][columns_with_target].values.ravel()
        pred_month_array_va = train_clinical_all.iloc[idx_va][columns_with_pred_month].values.ravel()
        score = smape_plus_1(y_true_array_va, calculate_predictions(pred_month_array_va, trend, target))
        print(f"{target} fold {f}: {score:.2f}")
        score_list.append(score)
    score = np.array(score_list).mean()
    print(f"{target}                 {score:.2f}")
    all_score_list.append(score)
    
print(f"cv score                       {np.array(all_score_list).mean():.2f}")

In [ ]:
feature = 'O15240'
# feature = 'O43505'
# display(train_clinical_all[feature])
# print(train_clinical_all[feature].quantile(0.25))
quantiles = [0, 0.05, 0.95, 1.0]

df_plot = []
for quantile_low, quantile_high in tqdm(zip(quantiles[:-1], quantiles[1:])):
    item = {
        'quantile_low': quantile_low,
        'quantile_high': quantile_high,
        'quantile_middle': (quantile_low + quantile_high) / 2
    }
    quantile_low_value = train_clinical_all[feature].quantile(quantile_low)
    quantile_high_value = train_clinical_all[feature].quantile(quantile_high)
    item['quantile_low_value'] = quantile_low_value
    item['quantile_high_value'] = quantile_high_value
    
    if quantile_high == 1:
        quantile_high_value += 0.00001
        
    train_clinical_all_filtered = train_clinical_all[
        (train_clinical_all[feature] >= quantile_low_value)
        & (train_clinical_all[feature] < quantile_high_value)
    ]
    for target in ['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']:
        item[f'{target}_shift'] = find_best_const(train_clinical_all_filtered, target)
    df_plot.append(item)
    
df_plot = pd.DataFrame(df_plot)

In [ ]:
npx_groups = [
    {'quantile_low': 0.0, 'quantile_high': 0.05},
    {'quantile_low': 0.05, 'quantile_high': 0.95},
    {'quantile_low': 0.95, 'quantile_high': 1.0},
]
target_to_npx_groups_shift = defaultdict(list)

for target in ['updrs_1', 'updrs_2', 'updrs_3']:
    for npx_group in npx_groups:
        item = npx_group.copy()
        item['feature'] = feature
        
        if item['quantile_low'] == 0:
            item['quantile_low_value'] = -np.inf
        else:
            item['quantile_low_value'] = train_clinical_all[feature].quantile(item['quantile_low'])
            
        if item['quantile_high'] == 1:
            item['quantile_high_value'] = np.inf
        else: 
            item['quantile_high_value'] = train_clinical_all[feature].quantile(item['quantile_high'])

        train_clinical_all_filtered = train_clinical_all[
            (train_clinical_all[feature] >= item['quantile_low_value'])
            & (train_clinical_all[feature] < item['quantile_high_value'])
        ]
        
        item['shift'] = find_best_const(train_clinical_all_filtered, target)
        target_to_npx_groups_shift[target].append(item)